# Adapt a dstl network to be used with more or fewer classes
Based on the experience that a network on the 6 classes (0, 1, 4, 7, 8, 9) has trouble finding the less frequent classes (7, 8 ,9), we consider using this network as a basis for a new network that learns only the less frequent classes.

Aborted due to time constraint

In [ ]:
import os
import sys
import pprint

import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)

# need to point the correct location of the py-faster-rcnn version of the caffe library
add_path('/home/ubuntu/src/py-faster-rcnn/caffe-fast-rcnn/python')
add_path('/home/ubuntu/src/py-faster-rcnn/lib')
import caffe
from datasets.factory import get_imdb, list_imdbs
from fast_rcnn.test import test_net
from fast_rcnn.train import get_training_roidb, train_net
from fast_rcnn.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
print "Loaded caffe version {:s} from {:s}.".format(caffe.__version__, caffe.__path__[0])

In [ ]:
arch = 'VGG16'
target_classes = (7, 8, 9) # classes that should be in the output network
old_classes = (0, 1, 4, 7, 8, 9) # classes of the network from which we take the original weights
new_classes = () # classes in the new network (that will overwrite the old)

def create_ix(classes):
    appendix = '' # This will codify which classes to train (if not all, in which case this string should be empty)
    infix = '.' # for directories
    if classes is not None:
        appendix = '_'+''.join(['{:d}'.format(c) for c in classes])
        infix = '{:d}'.format(len(classes))+'_classes'
    return infix, appendix
        
target_infix, target_appendix = create_ix(target_classes)
old_infix, old_appendix = create_ix(old_classes)
new_infix, new_appendix = create_ix(new_classes)

Load the old (and new) model with weights and the target model without weights.

In [ ]:
target_test_file = '/home/ubuntu/dstl/models/{:s}/{:s}/test.prototxt'.format(arch, target_infix)
target_net = caffe.Net(target_test_file, caffe.TEST)

old_test_file = '/home/ubuntu/dstl/models/{:s}/{:s}/test.prototxt'.format(arch, old_infix)
old_model = '/data/output/faster_rcnn_end2end/dstl{:s}_train/{:s}_faster_rcnn_third_iter_1000.caffemodel'.format(old_appendix, arch.lower())
old_net = caffe.Net(old_test_file, old_model, caffe.TEST)
if len(new_classes) > 0:
    new_test_file = '/home/ubuntu/dstl/models/{:s}/{:s}/test.prototxt'.format(arch, new_infix)
    new_model = '/data/output/faster_rcnn_end2end/dstl{:s}_train/{:s}_faster_rcnn_third_iter_1000.caffemodel'.format(new_appendix, arch.lower())
    new_net = caffe.Net(new_test_file, new_model, caffe.TEST)

Copy the layer parameters from the original network to the new, discarding any parameters that do not 'fit' in the new network.

In [ ]:
def copy_data(layer_name):
    """
    inputs:
        layer_name: which layer to copy
    outputs:
        None
    """
    use_new = len(new_classes) > 0:
    for c in target_classes:
        for i in range(2): # weights and biases
            target_param = target_net.params[layer_name][i]
            old_param = old_net.params[layer_name][i]
            new_param = None
            if use_new:
                new_param = new_net.params[layer_name][i]
                
            target_shape = target_param.data.shape
            if len(target_shape) > 2:
                
            

In [ ]:
for layer_name in target_net._layer_names:
    try:
        if layer_name == 'cls_score' or layer_name == 'bbox_pred':
            copy_data(layer_name)
        else:
            for old_param, target_param in zip(old_net.params[layer_name], target_net.params[layer_name]):
                target_param.data[...] = old_param.data
    except KeyError:
        pass

Check to see if the new and original networks are indeed identitical up to the discarding of weights. Unforunately this does not take into account error loaded into `orig_net` due to the absense of layers in the caffemodel weights file.

Save the output

In [ ]:
filename = '/home/ubuntu/dstl/models/{:s}/{:s}/trained/adapted_for{:s}_from{:s}{:s}.caffemodel'.format(arch, target_infix, target_appendix, old_appendix, new_appendix)
# target_net.save(filename)
print 'Saved weights file at {:s}'.format(filename)

It appears one layer, `rpn_conv/3x3` is not properly initialized from the weights file, so let's inspect the weights file and the network a little more...

This seems to be an error on the py_faster_rcnn side.

Load the .caffemodel file *without* specifying the prototxt